## Smart E - Learning Framework Using Deep Q - Network of Reinforcement Learning

### Installing the Necessary Libraries

In [ ]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install swig
!pip install gymnasium[box2d]

### Importing the Libraries

In [50]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import tensorflow as tf
import gymnasium as gym
from gymnasium import spaces

from collections import deque, namedtuple

from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Input
from keras.losses import MSE
from keras.optimizers import Adam

### Defining the States for the Environment

In [51]:
states = ['Start', 'Reading', 'Watching', 'Entertaining', 'Getting Bored', 'Rest', 'Writing', 'Playing', 'Clicking Ad', 'Course Completion', 'Quit']

for i in range(len(states)) :
    print("State {}: {}".format(i+1, states[i]))

State 1: Start
State 2: Reading
State 3: Watching
State 4: Entertaining
State 5: Getting Bored
State 6: Rest
State 7: Writing
State 8: Playing
State 9: Clicking Ad
State 10: Course Completion
State 11: Quit


### Defining the Actions for the Environment

In [52]:
actions = ['Stay', 'Previous Course', 'Additional Content', 'Do assignment', 'Move to Exams', 'Move to high-level course', 'Move to low-level course', 'Move to Social Media']

for i in range(len(actions)) :
    print("Action {}: {}".format(i+1, actions[i]))

Action 1: Stay
Action 2: Previous Course
Action 3: Additional Content
Action 4: Do assignment
Action 5: Move to Exams
Action 6: Move to high-level course
Action 7: Move to low-level course
Action 8: Move to Social Media


### Assigning Rewards for each action to be taken respectively

In [53]:
rewards = {}
rewardPoints = [10, 20, 50, 60, 100, 100, 70, -10]

for i in range(len(actions)) :
    rewards['{}'.format(actions[i])] = rewardPoints[i]

keys = list(rewards.keys())
for i in range(len(rewards)) :
    print("Action: {}\tReward Points: {}".format(keys[i], rewards.get(keys[i])))

Action: Stay	Reward Points: 10
Action: Previous Course	Reward Points: 20
Action: Additional Content	Reward Points: 50
Action: Do assignment	Reward Points: 60
Action: Move to Exams	Reward Points: 100
Action: Move to high-level course	Reward Points: 100
Action: Move to low-level course	Reward Points: 70
Action: Move to Social Media	Reward Points: -10


### Customizing the State values for the Custom Environment

In [55]:
customStates = {}

currentStateNumber = 0.0

for state in states :
    customStates['{}'.format(state)] = currentStateNumber
    currentStateNumber += 1

for key in list(customStates.keys()) :
    print("State: {}\tAssigned State Number: {}".format(key, customStates[key]))

finalStateForm = np.array(list(customStates.values())).astype(np.float32)

State: Start	Assigned State Number: 0.0
State: Reading	Assigned State Number: 1.0
State: Watching	Assigned State Number: 2.0
State: Entertaining	Assigned State Number: 3.0
State: Getting Bored	Assigned State Number: 4.0
State: Rest	Assigned State Number: 5.0
State: Writing	Assigned State Number: 6.0
State: Playing	Assigned State Number: 7.0
State: Clicking Ad	Assigned State Number: 8.0
State: Course Completion	Assigned State Number: 9.0
State: Quit	Assigned State Number: 10.0


### Creating and Registering the Environment in Gymnasium

In [82]:
class ELearningEnv(gym.Env) :
    def __init__(self) :
        super(ELearningEnv, self).__init__()
        self.states = states
        self.actions = actions
        self.rewards = rewards
        self.action_space = spaces.Discrete(len(self.actions))
        self.observation_space = spaces.Discrete(len(self.states))
        self.current_state = None

    def reset(self) :
        self.current_state = 0
        return self.current_state

    def step(self, action) :
        reward = self.rewards.get(self.actions[action], 0)
        done = (self.states[self.current_state] in ['Course Completion', 'Quit'])
        if not done :
            if action == 0 :
                pass
            elif action == 1 :
                self.current_state = max(0, self.current_state - 1)
            elif action == 2 :
                self.current_state = min(len(self.states) - 1, self.current_state + 1)
            elif action == 3 :
                self.current_state = min(len(self.states) - 2, self.current_state + 2)
            elif action == 4 :
                self.current_state = min(len(self.states) - 3, self.current_state + 3)
            elif action == 5 :
                self.current_state = min(len(self.states) - 4, self.current_state + 4)
            elif action == 6 :
                self.current_state = min(len(self.states) - 5, self.current_state + 5)
            elif action == 7 :
                self.current_state = min(len(self.states) - 6, self.current_state + 6)
        return self.current_state, reward, done, {}

    def render(self, mode = "human") :
        print(f"Current State: {self.states[self.current_state]}")

gym.register(id="ELearningEnv-v1", entry_point="__main__:ELearningEnv")

/home/sundhar-sg/anaconda3/envs/tensorflow/lib/python3.11/site-packages/gymnasium/envs/registration.py:693: UserWarning: WARN: Overriding environment ELearningEnv-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


### Setting the Seeding value for the Environment

In [57]:
tf.random.set_seed(0)

### Defining the Hyperparameters

In [58]:
MEMORY_SIZE = 100_000
GAMMA = 0.995
ALPHA = 1e-3
NUM_STEPS_FOR_EPISODE = 4

### Defining the Environment for use

In [83]:
env = gym.make("ELearningEnv-v1")
env.reset()

state_size = env.observation_space.n
action_size = env.action_space.n

print("Number of States Available: ", state_size, "\nNumber of Actions: ", action_size)

Number of States Available:  11 
Number of Actions:  8


/home/sundhar-sg/anaconda3/envs/tensorflow/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:197: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/home/sundhar-sg/anaconda3/envs/tensorflow/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:210: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
/home/sundhar-sg/anaconda3/envs/tensorflow/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:218: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'int'>`
  logger.warn(


### Reset the Environment

In [84]:
current_state = env.reset()

### Test the Environment for a action

In [85]:
action = 4

next_state, reward, done, _ = env.step(action)

env.render()

Current State: Entertaining


/home/sundhar-sg/anaconda3/envs/tensorflow/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:242: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


### Create the Deep Neural Network for the Q - Network

In [70]:
QNetwork = Sequential(
    [
        Input(shape=state_size),
        Dense(units=64, activation="relu"),
        Dense(units=128, activation="relu"),
        Dense(units=64, activation="relu"),
        Dense(units=action_size, activation="linear")
    ]
)

target_QNetwork = Sequential(
    [
        Input(shape=state_size),
        Dense(units=64, activation="relu"),
        Dense(units=128, activation="relu"),
        Dense(units=64, activation="relu"),
        Dense(units=action_size, activation="linear")
    ]
)

optimizer = Adam(learning_rate=ALPHA)

### Implementing Experience Replay

In [71]:
experience = namedtuple(typename="Experience", field_names=["state", "action", "reward", "next_state", "done"])

### Defining Compute Loss function for the Deep Q - Learning with Experience Replay

In [72]:
def compute_loss(experiences, gamma, QNetwork, target_QNetwork) :

    # Unpack the mini-batch of experiences
    states, actions, rewards, next_states, done_vals = experiences

    # Compute max Q^(s,a)
    max_qsa = tf.reduce_max(input_tensor=target_QNetwork(next_states), axis=-1)

    # Set y = R if episode terminates, otherwise set y = R + gamma max Q^(s,a)
    yTargets = rewards + (gamma * max_qsa * (1 - done_vals))

    # Get the q_values and reshape to match y_targets
    q_values = QNetwork(states)
    q_values = tf.gather_nd(q_values, tf.stack([tf.range(q_values.shape[0]),
                                                tf.cast(actions, tf.int32)], axis = 1))
    
    # Compute the Loss
    loss = MSE(yTargets, q_values)

    return loss

### Update the Target Network Weights

In [73]:
TAU = 1e-3 # Soft update parameter

def update_target_network(QNetwork, target_QNetwork) :
    for targetWeights, QNetWeights in zip(target_QNetwork.weights, QNetwork.weights) :
        targetWeights.assign(TAU * QNetWeights + (1.0 - TAU) * targetWeights)

### Getting the Action based on Epsilon-greedy policy

In [74]:
import random

def get_action(q_values, epsilon=0.0) :

    # Returns an action based on the Epsilon-greedy policy

    if random.random() > epsilon :
        return np.argmax(q_values.numpy()[0])
    return random.choice(np.arange(8))

### Checking the Update conditions for the network

In [75]:
MINIBATCH_SIZE = 64

def check_update_conditions(t, num_steps_upd, memory_buffer) :

    # Returns a boolean that will be True if the conditions are met and False otherwise.

    if (t + 1) % num_steps_upd == 0 and len(memory_buffer) > MINIBATCH_SIZE :
        return True
    return False

### Getting the Experiences Randomly from the Memory Buffer

In [76]:
def get_experiences(memory_buffer) :

    # Returns a random sample of experience tuples drawn from the memory buffer.
    experiences = random.sample(memory_buffer, k=MINIBATCH_SIZE)
    states = tf.convert_to_tensor(
        np.array([e.state for e in experiences if e is not None]), dtype=tf.float32
    )
    actions = tf.convert_to_tensor(
        np.array([e.action for e in experiences if e is not None]), dtype=tf.float32
    )
    rewards = tf.convert_to_tensor(
        np.array([e.reward for e in experiences if e is not None]), dtype=tf.float32
    )
    next_states = tf.convert_to_tensor(
        np.array([e.next_state for e in experiences if e is not None]), dtype=tf.float32
    )
    done_vals = tf.convert_to_tensor(
        np.array([e.done for e in experiences if e is not None]).astype(np.uint8),
        dtype=tf.float32,
    )
    return (states, actions, rewards, next_states, done_vals)

### Updating the Epsilon value

In [77]:
EPSILON_MIN = 0.01 # Minimum Epsilon value for the Epsilon-greedy policy
EPSILON_DECAY = 0.995 # Epsilon-decay rate for the Epsilon-greedy policy

def get_new_epsilon(epsilon) :

    # Returns the updated Epsilon value for the Epsilon-greedy policy

    return max(EPSILON_MIN, EPSILON_DECAY * epsilon)

### Update the Network Weights

In [78]:
@tf.function
def agent_learn(experiences, gamma) :
    
    # Calculate the loss
    with tf.GradientTape() as tape :
        loss = compute_loss(experiences, gamma, QNetwork, target_QNetwork)

    # Get the Gradients of the loss with respect to the weights
    gradients = tape.gradient(loss, QNetwork.trainable_variables)

    # Update the weights of the QNetwork
    optimizer.apply_gradients(zip(gradients, QNetwork.trainable_variables))

    # Update the Weights of the Target QNetwork
    update_target_network(QNetwork, target_QNetwork)

### Training the Agent

In [81]:
import time

start = time.time()

numberOfEpisodes = 100
maximumNumberTimesteps = 500

totalPointHistory = []

numberPointAverage = 10 # Number of total points to use for averaging
epsilon = 1.0 # Initial Epsilon value for Epsilon-greedy policy

# Create a memory buffer D with capacity N
memory_buffer = deque(maxlen=MEMORY_SIZE)

# Set the target network weights equal to the Q-Network weights
target_QNetwork.set_weights(QNetwork.get_weights())

for i in range(numberOfEpisodes) :

    # Reset the environment to the initial state and get the initial state
    state = env.reset()
    totalPoints = 0

    for t in range(maximumNumberTimesteps) :

        # From the current state S, choose an action A using an Epsilon-greedy policy
        state_qn = np.expand_dims(state, axis=0) # State needs to be the right shape for the Q-Network
        q_values = QNetwork(state_qn)
        action = get_action(q_values, epsilon)

        # Take action A and receive reward R and the next state S'
        next_state, reward, done, _ = env.step(action)

        # Store experience tuple (S,A,R,S') in the memory buffer
        # Storing the done variable for our convenience
        memory_buffer.append(experience(state, action, reward, next_state, done))

        # Update the network only after every NUM_STEPS_FOR_UPDATE timesteps.
        update = check_update_conditions(t, NUM_STEPS_FOR_EPISODE, memory_buffer)

        if update :

            # Sample random mini-batch of experience tuples (S,A,R,S') from D
            experiences = get_experiences(memory_buffer)

            # Set the y-targets, perform a gradient descent step,
            # and update the network weights"
            agent_learn(experiences, GAMMA)

        state = next_state.copy()
        totalPoints += reward

        if done :
            break

    totalPointHistory.append(totalPoints)
    averageLatestPoints = np.mean(totalPointHistory[-numberPointAverage:])

    # Update the Epsilon value
    epsilon = get_new_epsilon(epsilon)

    print(f"\rEpisode {i+1} | Total point average of the last {numberPointAverage} episodes: {averageLatestPoints:.2f}", end="")

    if (i + 1) % numberPointAverage == 0 :
        print(f"\rEpisode {i+1} | Total point average of the last {numberPointAverage} episodes: {averageLatestPoints:.2f}")

totalTimeTaken = time.time() - start

print(f"Total Runtime of the Agent Training: {totalTimeTaken:.2f} s ({(totalTimeTaken / 60):.2f} mins)")


ValueError: Exception encountered when calling layer 'sequential_2' (type Sequential).

Input 0 of layer "dense_8" is incompatible with the layer: expected min_ndim=2, found ndim=1. Full shape received: (1,)

Call arguments received by layer 'sequential_2' (type Sequential):
  • inputs=tf.Tensor(shape=(1,), dtype=float32)
  • training=None
  • mask=None